In [1]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle
import datetime
import py7zr

db = pd.DataFrame()
readPath = r'D:\work\project 3 event study\2020_new\2020\***'
dataPathLs = np.array(glob.glob(readPath))        
for i in dataPathLs:
    df = pd.read_csv(i, compression='gzip', encoding="GBK")
    if df[(df["名称"].str.contains("九章资产"))|(df["名称"].str.contains("明汯"))|(df["名称"].str.contains("灵均"))|
         (df["名称"].str.contains("金锝"))|(df["名称"].str.contains("九坤"))|(df["名称"].str.contains("致诚卓远"))].shape[0] != 0:
        db = pd.concat([db, df[(df["名称"].str.contains("九章资产"))|(df["名称"].str.contains("明汯"))|(df["名称"].str.contains("灵均"))|
         (df["名称"].str.contains("金锝"))|(df["名称"].str.contains("九坤"))|(df["名称"].str.contains("致诚卓远"))]])        

db["FundName"] = np.where(db["名称"].str.contains("九章资产"), "九章", np.where(
db["名称"].str.contains("明汯"), "明汯", np.where(
db["名称"].str.contains("灵均"), "灵均", np.where(
db["名称"].str.contains("金锝"), "金锝", np.where(
db["名称"].str.contains("九坤"), "九坤", "致诚卓远")))))     

db = db[["FundName", "名称","截止日", "StockID", "StockName", "股数", "占总股本比例(%)"]]
db = db.rename(columns={"名称":"ProductName", "截止日":"Date", "股数":"HoldingShare", "占总股本比例(%)":"HoldingRatio"})
print(db)       

   FundName                     ProductName      Date   StockID StockName  \
18       明汯     上海明汯投资管理有限公司-明汯价值成长1期私募投资基金  20200630  SH600197     伊力特     
23       明汯     上海明汯投资管理有限公司-明汯价值成长1期私募投资基金  20200930  SH600213      亚星客车   
6        九章          浙江九章资产管理有限公司-幻方星月石私募基金  20200331  SH600318      新力金融   
18       明汯     上海明汯投资管理有限公司-明汯价值成长1期私募投资基金  20200630  SH600361      华联综超   
4        九章        浙江九章资产管理有限公司-幻方星月石7号私募基金  20200331  SH600371      万向德农   
..      ...                             ...       ...       ...       ...   
7        明汯    上海明汯投资管理有限公司-明汯朱庇特2号私募证券投资基金  20200930  SZ300875      捷强装备   
3        明汯    上海明汯投资管理有限公司-明汯朱庇特2号私募证券投资基金  20200930  SZ300892      品渥食品   
1        明汯    上海明汯投资管理有限公司-明汯朱庇特2号私募证券投资基金  20200930  SZ300893      松原股份   
5        明汯  上海明汯投资管理有限公司-明汯中性添利13号私募证券投资基金  20200930  SZ300893      松原股份   
2        明汯    上海明汯投资管理有限公司-明汯朱庇特2号私募证券投资基金  20200930  SZ300895      铜牛信息   

    HoldingShare  HoldingRatio  
18     2109827.0        0.4856  
23     10

In [6]:
db1 = pd.merge(db, dd[dd["Date"] == 20201105][["StockID", "close"]], on="StockID", how="left")
db1["MarketValue"] = db1["close"]*db1["HoldingShare"]/10000
db1["year"] = np.where(db1["Date"] > 20200000, 2020, 2019)
sl = db1[db1["Date"]>20200000]["StockID"].unique()
db1 = pd.merge(db1, dd[(dd["StockID"].isin(sl)) & (dd["Date"]<=20200930)].groupby(["year", "StockID"])["ret"].sum().reset_index(), on=["year", "StockID"], how="left")
db1 = db1.rename(columns={"ret":"Year2Date(%)"})
db1 = pd.merge(db1, dd[(dd["StockID"].isin(sl)) & (dd["Date"]>20200930)].groupby(["year", "StockID"])["ret"].sum().reset_index(), on=["year", "StockID"], how="left")
db1 = db1.rename(columns={"ret":"CumReturn2Now(%)"})
db1["Universe"] = np.where(db1["StockID"].isin(IF), "IF", np.where(
db1["StockID"].isin(IC), "IC", np.where(
db1["StockID"].isin(CSI1000), "CSI1000", np.where(
db1["StockID"].isin(CSIRest), "CSIRest", "Rest"))))
db1 = db1[["FundName", "ProductName", "Date", "StockID", "StockName", "Universe", "HoldingShare", "HoldingRatio", "MarketValue", "CumReturn2Now(%)", "Year2Date(%)"]]
db1 = db1.sort_values(by=["Date", "FundName"], ascending=False)

In [12]:
50.4 * 299417 / 10000

1509.0616799999998

In [11]:
db1 = db1[db1['Date'] == 20200930]
db1.to_csv(r'D:\work\project 3 event study\2020_new\heavyloading.csv', index=False, encoding='GBK')

In [3]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle
import datetime
import py7zr

def clean(x):
    cl_ean = x["ListDays"] != 0
    x = x[cl_ean]
    cl_ean1 = x["trade_stats"] == 1
    x = x[cl_ean1]
    return x.iloc[:,[0,1,2,3,5,6,7,8,9,10,11,12,13]]

startDate = "20200101"
endDate = "20201105"
path = r'F:\Download\StockFactors'
all_files = np.array(glob.glob(os.path.join(path, "*.csv")))
dateLs = np.array([os.path.basename(i).split('_')[1].split('.')[0] for i in all_files])
all_files = all_files[(dateLs >= startDate) & (dateLs <= endDate)]

dd = clean(pd.read_csv(all_files[0], encoding="GBK"))
for i in range(1, len(all_files)):
    dn = clean(pd.read_csv(all_files[i], encoding="GBK"))
    dd = pd.concat([dd, dn], axis = 0, ignore_index = True)
dd = dd.sort_values(by=["Date"])
dd = dd.rename(columns={"Symbol":"StockID"})
dd["year"] = 2020
display(dd)

,Date,StockID,Name,ListDays,Totalshares,Marketshares,TotalValue,MarketValue,close,open,ret,volume,amt,year
0,20200102,SH600000,浦发银行,4765,2.935208e+10,2.810376e+10,3.660204e+07,3.504539e+07,12.47,12.47,0.808407,51629079,6.474462e+08,2020
2486,20200102,SZ002479,富春环保,2209,8.888880e+08,8.821442e+08,6.248883e+05,6.201474e+05,7.03,7.06,0.285307,15363093,1.080193e+08,2020
2487,20200102,SZ002480,新筑股份,2218,6.510260e+08,6.399968e+08,2.604104e+05,2.559987e+05,4.00,3.98,1.010101,5142040,2.051803e+07,2020
2488,20200102,SZ002481,双塔食品,2121,1.243390e+09,1.107128e+09,1.024553e+06,9.122732e+05,8.24,7.99,3.778338,42034674,3.442204e+08,2020
2489,20200102,SZ002482,广田集团,2200,1.537280e+09,1.529295e+09,6.963877e+05,6.927705e+05,4.53,4.42,3.424658,16237157,7.333702e+07,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780122,20201105,SH603699,纽威股份,1646,7.500000e+08,7.500000e+08,1.088250e+06,1.088250e+06,14.51,14.49,1.539538,2934203,4.224227e+07,2020
780123,20201105,SH603700,宁水集团,433,2.032420e+08,1.033233e+08,5.938731e+05,3.019107e+05,29.22,29.26,0.793377,2377317,6.913123e+07,2020
780124,20201105,SH603701,德宏股份,1089,2.626926e+08,2.613619e+08,2.540238e+05,2.527370e+05,9.67,9.70,0.415369,2437371,2.355265e+07,2020
780111,20201105,SH603683,晶华新材,741,1.266700e+08,1.266700e+08,2.050787e+05,2.050787e+05,16.19,15.96,2.274163,3824145,6.184329e+07,2020


In [4]:
import pymongo
import pandas as pd
import numpy as np
import pickle
import datetime
import time
import gzip
import lzma
import pytz

def DB(host, db_name, user, passwd):
    auth_db = db_name if user not in ('admin', 'root') else 'admin'
    url = 'mongodb://%s:%s@%s/?authSource=%s' % (user, passwd, host, auth_db)
    client = pymongo.MongoClient(url, maxPoolSize=None)
    db = client[db_name]
    return db

def read_memb_daily(db, name, start_date=None, end_date=None, skey=None, index_id=None, interval=None, col=None, return_sdi=True):
    collection = db[name]
    # Build projection
    prj = {'_id': 0}
    if col is not None:
        if return_sdi:
            col = ['skey', 'date', 'interval'] + col
        for col_name in col:
            prj[col_name] = 1

    # Build query
    query = {}
    if skey is not None:
        query['skey'] = {'$in': skey}
    if index_id is not None:
        query['index_id'] = {'$in': index_id}
    if interval is not None:
        query['interval'] = {'$in': interval}
    if start_date is not None:
        if end_date is not None:
            query['date'] = {'$gte': start_date, '$lte': end_date}
        else:
            query['date'] = {'$gte': start_date}
    elif end_date is not None:
        query['date'] = {'$lte': end_date}

    # Load data
    cur = collection.find(query, prj)
    df = pd.DataFrame.from_records(cur)
    if df.empty:
        df = pd.DataFrame()
    else:
        df = df.sort_values(by=['date', 'index_id', 'skey'])
    return df    

def build_query(start_date=None, end_date=None, index_id=None):
    query = {}
    def parse_date(x):
        if type(x) == int:
            return x
        elif type(x) == str:
            if len(x) != 8:
                raise Exception("date must be YYYYMMDD format")
            return int(x)
        elif type(x) == datetime.datetime or type(x) == datetime.date:
            return x.strftime("%Y%m%d").astype(int)
        else:
            raise Exception("invalid date type: " + str(type(x)))
    if start_date is not None or end_date is not None:
        query['date'] = {}
        if start_date is not None:
            query['date']['$gte'] = parse_date(start_date)
        if end_date is not None:
            query['date']['$lte'] = parse_date(end_date)
    def parse_symbol(x):
        if type(x) == int:
            return x
        else:
            return int(x)
    if index_id:
        if type(index_id) == list or type(index_id) == tuple:
            query['index_id'] = {'$in': [parse_symbol(x) for x in index_id]}
        else:
            query['index_id'] = parse_symbol(index_id)
    return query

def write_memb_data(db, name, df):
    collection = db[name]
    df1 = []
    for symbol in df['index_id'].unique():
        if symbol in collection.distinct('index_id'):
            symbol = int(symbol)
            m_ax = pd.DataFrame.from_records(collection.find({'index_id':{'$in':[symbol]}}).sort([('date',-1)]).skip(0).limit(1))['date'].values[0]
            df2 = df[(df['index_id'] == symbol) & (df['date'] > m_ax)]
            print(df2)
            df1 += [df2]
        else:
            print(symbol)
            df2 = df[(df['index_id'] == symbol)]
            print(df2)
            df1 += [df2]
    df1 = pd.concat(df1).reset_index(drop=True)
    df1 = df1.to_dict('records')
    collection.insert_many(df1) 

def delete_memb_data(db, name, start_date=None, end_date=None, index_id=None):
    collection = db[name]
    query = build_query(start_date, end_date, index_id)
    if not query:
        print('cannot delete the whole table')
        return None
    collection.delete_many(query)  


database_name = 'com_md_eq_cn'
user = "zhenyuy"
password = "bnONBrzSMGoE"

pd.set_option('max_columns', 200)
db1 = DB("192.168.10.178", database_name, user, password)

In [5]:
IC = read_memb_daily(db1, 'index_memb', 20201105, 20201105, index_id=[1000905])
IC['skey'] = np.where(IC['skey'] > 2000000, 'SZ' + (IC['skey']-2000000).astype(str), 'SH' + (IC['skey']-1000000).astype(str))
IC =IC['skey'].unique()
IF = read_memb_daily(db1, 'index_memb', 20201105, 20201105, index_id=[1000300])
IF['skey'] = np.where(IF['skey'] > 2000000, 'SZ' + (IF['skey']-2000000).astype(str), 'SH' + (IF['skey']-1000000).astype(str))
IF =IF['skey'].unique()
CSI1000 = read_memb_daily(db1, 'index_memb', 20201105, 20201105, index_id=[1000852])
CSI1000['skey'] = np.where(CSI1000['skey'] > 2000000, 'SZ' + (CSI1000['skey']-2000000).astype(str), 'SH' + (CSI1000['skey']-1000000).astype(str))
CSI1000 = CSI1000['skey'].unique()
CSIRest = read_memb_daily(db1, 'index_memb', 20201105, 20201105, index_id=[1000985])
CSIRest['skey'] = np.where(CSIRest['skey'] > 2000000, 'SZ' + (CSIRest['skey']-2000000).astype(str), 'SH' + (CSIRest['skey']-1000000).astype(str))
CSIRest = CSIRest['skey'].unique()

In [ ]:
d = read_memb_daily(db1, 'index_memb', 20190101, 20201015, index_id=[1000905])

In [162]:
sto_ck = pd.read_csv('F:\\all_stock_201904_1.csv', encoding = "GBK").iloc[:,1:]
sto_ck["returns"] = sto_ck.groupby("Symbol")['close'].apply(lambda x: x/x.shift(1)-1)
sto_ck["Date"] = sto_ck["Date"].apply(lambda x: str(x))
sto_ck["Date"] = pd.to_datetime(sto_ck["Date"])
sto_ck

,Date,Symbol,Name,ListDays,Totalshares,Marketshares,TotalValue,MarketValue,close,open,ret,volume,amt,returns
0,2018-05-02,SH600000,浦发银行,4355,2.935208e+10,2.810376e+10,3.237534e+07,3.099845e+07,10.914744,11.211609,-4.995693,7.005774e+07,7.690502e+08,NaN
1,2018-05-03,SH600000,浦发银行,4356,2.935208e+10,2.810376e+10,3.205247e+07,3.068931e+07,10.805893,10.875162,-0.997280,3.503163e+07,3.780554e+08,-0.009973
2,2018-05-04,SH600000,浦发银行,4357,2.935208e+10,2.810376e+10,3.178830e+07,3.043638e+07,10.716834,10.795998,-0.824176,2.248807e+07,2.417497e+08,-0.008242
3,2018-05-07,SH600000,浦发银行,4358,2.935208e+10,2.810376e+10,3.181766e+07,3.046448e+07,10.726729,10.706938,0.092336,2.771498e+07,2.971628e+08,0.000923
4,2018-05-08,SH600000,浦发银行,4359,2.935208e+10,2.810376e+10,3.214053e+07,3.077362e+07,10.835580,10.726729,1.014760,2.910053e+07,3.145377e+08,0.010148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885093,2019-05-13,SZ300778,新城市,2,8.000000e+07,2.000000e+07,3.464000e+05,8.660000e+04,43.300000,43.300000,10.010163,8.302000e+03,3.594766e+05,0.100102
885094,2019-05-14,SZ300778,新城市,3,8.000000e+07,2.000000e+07,3.810400e+05,9.526000e+04,47.630000,47.630000,10.000000,2.001600e+04,9.533621e+05,0.100000
885095,2019-05-15,SZ300778,新城市,4,8.000000e+07,2.000000e+07,4.191200e+05,1.047800e+05,52.390000,52.390000,9.993701,7.383100e+04,3.868006e+06,0.099937
885096,2019-05-16,SZ300778,新城市,5,8.000000e+07,2.000000e+07,4.248800e+05,1.062200e+05,53.110000,57.630000,1.374308,9.885354e+06,5.375233e+08,0.013743


In [148]:
len(CSIRest[CSIRest["date"] == "2019-12-31"]["ID"].unique())

1602

In [149]:
len(set(csi_rest.columns[(csi_rest.loc[csi_rest["Date"]=='20191231', :] != 0).values[0]][1:]) & set(CSIRest[CSIRest["date"] == "2019-12-31"]["ID"].unique()))

1602

In [104]:
csi_index = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000905.csv',
                        encoding="utf-8")
csi_index_300 = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000300.csv',
                            encoding="utf-8")
csi_index_1000 = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000852.csv',
                        encoding="utf-8")
csi_rest = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000985.csv',
                        encoding="utf-8")
csi_index["Date"] = csi_index["Date"].apply(lambda x: str(x))
csi_index_300["Date"] = csi_index_300["Date"].apply(lambda x: str(x))
csi_index_1000["Date"] = csi_index_1000["Date"].apply(lambda x: str(x))
csi_rest["Date"] = csi_rest["Date"].apply(lambda x: str(x))
da_te = csi_index["Date"]
sto_ck = pd.read_csv('F:\\all_stock_201910.csv', encoding = "GBK").iloc[:,1:]
sto_ck["returns"] = sto_ck.groupby("Symbol")['close'].apply(lambda x: x/x.shift(1)-1)
sto_ck["Date"] = sto_ck["Date"].apply(lambda x: str(x))
sto_ck["Date"] = pd.to_datetime(sto_ck["Date"])
xx = []
dd1 = []
dd2 = []
new_in = {}
old_out = {}
In_fromTop = {}
In_fromBottom = {}
Out_toTop = {}
Out_toBottom = {}
csi_rest = csi_rest[csi_rest["Date"] >= '20141103']
csi_index_300 = csi_index_300[csi_index_300["Date"] >= '20141103']
csi_index1 = csi_index[csi_index["Date"] >= '20141103']
csi_rest1 = {}
csi_rest1["Date"] = []
csi_rest1["Stocks"] = []
for i in range(len(csi_rest)):
    x1 = csi_rest.columns[csi_rest.iloc[i, :] != 0][1:]
    x2 = csi_index_300.columns[csi_index_300.iloc[i, :] != 0][1:]
    x3 = csi_index1.columns[csi_index1.iloc[i, :] != 0][1:]
    x4 = csi_index_1000.columns[csi_index_1000.iloc[i, :] != 0][1:]
    rx = list(((set(x1) - set(x2)) - set(x3)) - set(x4))
    csi_rest1["Date"].append(csi_rest.iloc[i, 0])
    csi_rest1["Stocks"].append(rx)
csi_rest1 = pd.DataFrame(csi_rest1)

xx.append(csi_rest1.iloc[0, 1])
for i in range(1, len(csi_rest)):
    xx.append(csi_rest1.iloc[i, 1])
    new_in[csi_rest1.iloc[i, 0]] = list(set(xx[i]) - set(xx[i - 1]))
    old_out[csi_rest1.iloc[i, 0]] = list(set(xx[i - 1]) - set(xx[i]))
if "SZ001914" in new_in["20200102"]:
    new_in["20200102"].remove("SZ001914")
    old_out["20200102"].remove("SZ000043")
if "SZ001872" in new_in["20200102"]:
    new_in["20200102"].remove("SZ001872")
    old_out["20200102"].remove("SZ000022")
new_in = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in new_in.items()]))
old_out = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in old_out.items()]))
new_in = new_in.dropna(axis=1, thresh=30)
old_out = old_out.dropna(axis=1, thresh=30)
print(new_in)
print(old_out)

hs300_list = csi_index_300.columns
zz500_list = csi_index.columns
zz1000_list = csi_index_1000.columns
for i in range(len(old_out.columns)):
    test1 = csi_index_300[csi_index_300["Date"] == old_out.columns[i]][list(set(hs300_list) & set(old_out.iloc[:, i]))]
    test2 = csi_index1[csi_index1["Date"] == old_out.columns[i]][list(set(zz500_list) & set(old_out.iloc[:, i]))]
    test3 = csi_index_1000[csi_index_1000["Date"] == old_out.columns[i]][list(set(zz1000_list) & set(old_out.iloc[:, i]))]
    Out_toTop[old_out.columns[i]] = list((set(test1.columns[(test1 != 0).values[0]]) | set(test2.columns[(test2 != 0).values[0]])) | set(test3.columns[(test3 != 0).values[0]]))
    Out_toBottom[old_out.columns[i]] = list(set(old_out.iloc[:, i]) - set(Out_toTop[old_out.columns[i]]))
for i in range(len(new_in.columns)):
    t = da_te[da_te[da_te == new_in.columns[i]].index[0] - 1]
    test1 = csi_index_300[csi_index_300["Date"] == t][list(set(hs300_list) & set(new_in.iloc[:, i]))]
    test2 = csi_index1[csi_index1["Date"] == t][list(set(zz500_list) & set(new_in.iloc[:, i]))]
    test3 = csi_index_1000[csi_index_1000["Date"] == t][list(set(zz1000_list) & set(new_in.iloc[:, i]))]
    In_fromTop[new_in.columns[i]] = list((set(test1.columns[(test1 != 0).values[0]]) | set(test2.columns[(test2 != 0).values[0]])) | set(test3.columns[(test3 != 0).values[0]]))
    In_fromBottom[new_in.columns[i]] = list(set(new_in.iloc[:, i]) - set(In_fromTop[new_in.columns[i]]))

Out_toTop = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in Out_toTop.items()]))
Out_toBottom = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in Out_toBottom.items()]))
In_fromTop = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in In_fromTop.items()]))
In_fromBottom = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in In_fromBottom.items()]))

     20141231  20150630  20151231  20160630  20161230  20170630  20171229  \
0    SH600870  SH603988  SZ300404  SZ300067  SH600236  SH603067  SZ000504   
1    SH600617  SZ300416  SZ002480  SH603123  SH600509  SH600081  SZ300678   
2    SH600139  SZ002637  SZ000504  SZ300259  SH600870  SZ002037  SZ002858   
3    SZ300081  SZ300391  SZ002587  SZ300490  SH600398  SZ300571  SZ002217   
4    SZ000888  SZ300309  SH600610  SZ000935  SZ000949  SH603429  SH603178   
..        ...       ...       ...       ...       ...       ...       ...   
348       NaN       NaN       NaN       NaN       NaN       NaN  SZ002774   
349       NaN       NaN       NaN       NaN       NaN       NaN  SH601199   
350       NaN       NaN       NaN       NaN       NaN       NaN  SZ002441   
351       NaN       NaN       NaN       NaN       NaN       NaN  SH600805   
352       NaN       NaN       NaN       NaN       NaN       NaN  SZ002172   

     20180629  20181228  20190701  20200102  
0    SZ002913  SH600592  SZ30

In [147]:
csi_rest[csi_rest["Date"] == "20191231"]

,Date,SH600004,SH600005,SH600006,SH600007,SH600011,SH600012,SH600018,SH600020,SH600022,...,SZ300762,SZ300763,SZ300767,SZ300769,SZ300772,SZ300778,SZ300779,SZ300780,SZ300781,SZ300786
2430,20191231,0.0,0.0,0.0,0.107114,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [141]:
Out_toBottom["20200102"][~Out_toBottom["20200102"].isnull()].values

array(['SH603779', 'SH603188', 'SH601258', 'SH603810', 'SH600319',
       'SZ002692', 'SZ000410'], dtype=object)

In [143]:
Out_toBottom["20190701"][~Out_toBottom["20190701"].isnull()].values

array(['SH600485', 'SZ002569', 'SH600726', 'SZ002220', 'SZ000504',
       'SZ300090', 'SZ000670', 'SZ300093', 'SZ000981', 'SH600615',
       'SZ000611', 'SZ300089', 'SH600186', 'SZ000868', 'SH600721',
       'SZ000897', 'SZ002336', 'SH600815', 'SZ300032', 'SZ002289',
       'SH600698', 'SZ300080', 'SZ002290', 'SZ002420', 'SZ002629',
       'SZ002190', 'SZ002072', 'SZ300167', 'SZ300163', 'SZ002200',
       'SZ002333', 'SH600666', 'SH600462', 'SZ300345', 'SZ300071',
       'SZ002499', 'SZ300063', 'SH600396', 'SZ000760', 'SH600891',
       'SH600112', 'SZ300116', 'SZ000572', 'SH600385', 'SZ000010',
       'SH600119', 'SZ000911', 'SZ002684', 'SZ000752', 'SZ300234',
       'SZ000806', 'SZ002021', 'SZ300312', 'SZ002175'], dtype=object)

In [163]:
st_list = sto_ck[(sto_ck["Name"].str.contains("ST") != False)  |
             (sto_ck["Name"].str.contains("退") != False)]["Symbol"].unique()
set(Out_toBottom["20190701"][~Out_toBottom["20190701"].isnull()].values) - set(set(st_list) & set(csi_rest.columns[(csi_rest.loc[csi_rest["Date"]=='20190628', :] != 0).values[0]][1:]))

{'SH600112',
 'SH600485',
 'SZ002220',
 'SZ300032',
 'SZ300063',
 'SZ300071',
 'SZ300080',
 'SZ300089',
 'SZ300090',
 'SZ300093',
 'SZ300116',
 'SZ300163',
 'SZ300167',
 'SZ300234',
 'SZ300312',
 'SZ300345'}